In [ ]:
## lo básico
import pandas as pd
import numpy as np

## visualización
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

## pre procesado
from sklearn.model_selection import train_test_split
#from ydata_profiling import ProfileReport

## modelado
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression


## métricas
from pandas.plotting import scatter_matrix
from sklearn.metrics import accuracy_score, auc, confusion_matrix, f1_score, precision_score, recall_score, roc_curve, roc_auc_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import label_binarize

## mejora de modelos
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score


## automatización
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# 1. Análisis exploratorio de los datos

In [121]:
url = "https://raw.githubusercontent.com/Valeriavinasl/ucm-tfm/main/data/Original_Encuesta.csv"

df_original = pd.read_csv(url, sep=",", encoding="utf-8", engine="python")
df_original.head()

,Marca temporal,Edad,Género,País de origen,¿Tienes alguna condición física o lesión que deba tenerse en cuenta?,"Si tienes alguna condición física o lesión que deba tenerse en cuenta, cuál es?",¿Qué nivel de experiencia tenías en senderismo o caminatas largas?,¿Cómo era tu condición física antes de empezar el Camino?,Motivo/s principal/es para hacer el Camino,¿Con cuántas personas hiciste el Camino?,...,¿Usaste transporte de mochila?,¿Reservaste alojamientos con antelación?,¿En qué tipo o tipos de alojamiento te hospedaste durante tu estancia?,¿Hace cuánto hiciste el Camino?,¿Volverías a hacer el Camino?,¿Cuál crees que es tu tipo ideal de etapa?,¿Qué nota le das a tu experiencia en el Camino? (0–10),¿Tuviste alguna lesión o problema físico durante el camino?,¿Qué consejos darías a alguien como tú antes de hacer el Camino?,Medio de transporte
0,2025-06-17 18:13:03.270,veintitrés,Mujer,España,No,NaN,Básica (≤ 10 km ocasionalmente),Media,"Religioso, Deportivo, Cultural/turístico, Pers...",En grupo pequeño (3–6 personas),...,No,Sí,"Albergues privados, Hostales / Hoteles",Más de 2 años,Sí,Medias (~20 km con algo de desnivel),9,"Sí, leve",NaN,NaN
1,2025-06-17 18:26:55.919,veintiocho,Hombre,España,NaN,NaN,Avanzada (rutas largas con mochila/montaña),Media,"Deportivo, Cultural/turístico, Personal/reflexivo",En grupo pequeño (3–6 personas),...,No,Sí,"Albergues públicos, Albergues privados, Hostal...",Más de 2 años,Sí,Medias (~20 km con algo de desnivel),9,No,Hacer etapas parecidas en distancia y terreno ...,NaN
2,2025-06-17 18:55:36.824,cincuenta y siete,Hombre,España,NaN,NaN,Moderada (10–20 km regularmente),Media,"Religioso, Cultural/turístico",En grupo organizado (>6),...,Sí,Sí,Hostales / Hoteles,Más de 2 años,Sí,Medias (~20 km con algo de desnivel),8,"Sí, leve",NaN,NaN
3,2025-06-17 20:45:31.528,veinticinco,Mujer,Colombia,NaN,Migrañas tensionales,Básica (≤ 10 km ocasionalmente),Media,Personal/reflexivo,En pareja,...,Sí,Sí,"Albergues privados, Hostales / Hoteles",Este año,Sí,Medias (~20 km con algo de desnivel),9,No,NaN,NaN
4,2025-06-17 21:12:37.526,21,Mujer,España,NaN,NaN,Básica (≤ 10 km ocasionalmente),Media,"Religioso, Cultural/turístico, Personal/reflexivo",En grupo organizado (>6),...,Sí,Sí,Albergues privados,En los últimos 2 años,Sí,Medias (~20 km con algo de desnivel),10,"Sí, leve",Prefiero no revelar y que cada uno viva su cam...,NaN


In [122]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 29 columns):
 #   Column                                                                           Non-Null Count  Dtype 
---  ------                                                                           --------------  ----- 
 0   Marca temporal                                                                   2000 non-null   object
 1   Edad                                                                             1999 non-null   object
 2   Género                                                                           2000 non-null   object
 3   País de origen                                                                   2000 non-null   object
 4   ¿Tienes alguna condición física o lesión que deba tenerse en cuenta?             1383 non-null   object
 5   Si tienes alguna condición física o lesión que deba tenerse en cuenta, cuál es?  364 non-null    object
 6   ¿Qué nivel de ex

In [123]:
df_original.isnull().sum()

,0
Marca temporal,0
Edad,1
Género,0
País de origen,0
¿Tienes alguna condición física o lesión que deba tenerse en cuenta?,617
"Si tienes alguna condición física o lesión que deba tenerse en cuenta, cuál es?",1636
¿Qué nivel de experiencia tenías en senderismo o caminatas largas?,0
¿Cómo era tu condición física antes de empezar el Camino?,0
Motivo/s principal/es para hacer el Camino,0
¿Con cuántas personas hiciste el Camino?,0


In [124]:
df_original.dtypes

,0
Marca temporal,object
Edad,object
Género,object
País de origen,object
¿Tienes alguna condición física o lesión que deba tenerse en cuenta?,object
"Si tienes alguna condición física o lesión que deba tenerse en cuenta, cuál es?",object
¿Qué nivel de experiencia tenías en senderismo o caminatas largas?,object
¿Cómo era tu condición física antes de empezar el Camino?,object
Motivo/s principal/es para hacer el Camino,object
¿Con cuántas personas hiciste el Camino?,object


In [125]:
df_original

,Marca temporal,Edad,Género,País de origen,¿Tienes alguna condición física o lesión que deba tenerse en cuenta?,"Si tienes alguna condición física o lesión que deba tenerse en cuenta, cuál es?",¿Qué nivel de experiencia tenías en senderismo o caminatas largas?,¿Cómo era tu condición física antes de empezar el Camino?,Motivo/s principal/es para hacer el Camino,¿Con cuántas personas hiciste el Camino?,...,¿Usaste transporte de mochila?,¿Reservaste alojamientos con antelación?,¿En qué tipo o tipos de alojamiento te hospedaste durante tu estancia?,¿Hace cuánto hiciste el Camino?,¿Volverías a hacer el Camino?,¿Cuál crees que es tu tipo ideal de etapa?,¿Qué nota le das a tu experiencia en el Camino? (0–10),¿Tuviste alguna lesión o problema físico durante el camino?,¿Qué consejos darías a alguien como tú antes de hacer el Camino?,Medio de transporte
0,2025-06-17 18:13:03.270,veintitrés,Mujer,España,No,NaN,Básica (≤ 10 km ocasionalmente),Media,"Religioso, Deportivo, Cultural/turístico, Pers...",En grupo pequeño (3–6 personas),...,No,Sí,"Albergues privados, Hostales / Hoteles",Más de 2 años,Sí,Medias (~20 km con algo de desnivel),9,"Sí, leve",NaN,NaN
1,2025-06-17 18:26:55.919,veintiocho,Hombre,España,NaN,NaN,Avanzada (rutas largas con mochila/montaña),Media,"Deportivo, Cultural/turístico, Personal/reflexivo",En grupo pequeño (3–6 personas),...,No,Sí,"Albergues públicos, Albergues privados, Hostal...",Más de 2 años,Sí,Medias (~20 km con algo de desnivel),9,No,Hacer etapas parecidas en distancia y terreno ...,NaN
2,2025-06-17 18:55:36.824,cincuenta y siete,Hombre,España,NaN,NaN,Moderada (10–20 km regularmente),Media,"Religioso, Cultural/turístico",En grupo organizado (>6),...,Sí,Sí,Hostales / Hoteles,Más de 2 años,Sí,Medias (~20 km con algo de desnivel),8,"Sí, leve",NaN,NaN
3,2025-06-17 20:45:31.528,veinticinco,Mujer,Colombia,NaN,Migrañas tensionales,Básica (≤ 10 km ocasionalmente),Media,Personal/reflexivo,En pareja,...,Sí,Sí,"Albergues privados, Hostales / Hoteles",Este año,Sí,Medias (~20 km con algo de desnivel),9,No,NaN,NaN
4,2025-06-17 21:12:37.526,21,Mujer,España,NaN,NaN,Básica (≤ 10 km ocasionalmente),Media,"Religioso, Cultural/turístico, Personal/reflexivo",En grupo organizado (>6),...,Sí,Sí,Albergues privados,En los últimos 2 años,Sí,Medias (~20 km con algo de desnivel),10,"Sí, leve",Prefiero no revelar y que cada uno viva su cam...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,2024-10-14 00:32:34.000,47,Mujer,México,No,NaN,Básica (≤ 10 km ocasionalmente),Media,"Cultural/turístico, Deportivo, Religioso",Solo/a,...,No,No,"Albergues públicos, Albergues privados",Más de 2 años,Sí,Medias (~20 km con algo de desnivel),8,No,Etapas de ~20–25 km funcionan bien; cuida la h...,A pie
1996,2024-12-07 17:53:18.000,24,Hombre,España,Sí,Fascitis plantar,Moderada (10–20 km regularmente),Media,"Cultural/turístico, Deportivo, Religioso",En pareja,...,No,Sí,"Albergues públicos, Albergues privados",Este año,Sí,Medias (~20 km con algo de desnivel),9,No,Etapas de ~20–25 km funcionan bien; cuida la h...,A pie
1997,2025-03-16 20:40:16.000,56,Hombre,España,No,NaN,Básica (≤ 10 km ocasionalmente),Alta,"Personal/reflexivo, Religioso",En grupo pequeño (3–6 personas),...,Sí,Sí,Hostales / Hoteles,Este año,Tal vez,Medias (~20 km con algo de desnivel),7,"Sí, leve",Etapas de ~20–25 km funcionan bien; cuida la h...,A pie
1998,2025-06-25 12:37:07.000,veintidós,Hombre,España,NaN,NaN,Moderada (10–20 km regularmente),Alta,"Deportivo, Religioso",En grupo pequeño (3–6 personas),...,No,No,Albergues privados,En los últimos 2 años,Sí,Medias (~20 km con algo de desnivel),9,No,NaN,A pie


Modificaciones

In [126]:
df_original = df_original.drop(columns=["Marca temporal"])

In [127]:
reenombre_v = {
    "Edad": "edad",
    "Género": "genero",
    "País de origen": "pais",
    "¿Tienes alguna condición física o lesión que deba tenerse en cuenta?": "is_condicion_fisica",
    "Si tienes alguna condición física o lesión que deba tenerse en cuenta, cuál es?": "detalle_condicion",
    "¿Qué nivel de experiencia tenías en senderismo o caminatas largas?": "nivel_expereincia_previo",
    "¿Cómo era tu condición física antes de empezar el Camino?": "condicion_previa",
    "Motivo/s principal/es para hacer el Camino": "motivo",
    "¿Con cuántas personas hiciste el Camino?": "n_acompañantes",
    "¿Qué época del año lo hiciste?": "epoca",
    "¿Habías hecho el camino anteriormente?": "is_camino_realizado_prev",
    "¿Qué ruta del Camino hiciste?": "ruta",
    "¿Cuántos días te llevó completarlo?": "dias",
    "¿Desde donde lo empezaste?": "inicio",
    "¿Qué distancia total caminaste (aproximadamente)?": "distancia",
    "¿Qué tan duras fueron las etapas que hiciste en general?": "dureza",
    "¿Qué tipo de calzado usaste principalmente durante el Camino?": "calzado",
    "¿Tu calzado fue adecuado para el tipo de terreno y etapas?": "is_calzado_adecuado",
    "¿Usaste transporte de mochila?": "is_tr_mochila",
    "¿Reservaste alojamientos con antelación?": "is_reserva_aloj",
    "¿En qué tipo o tipos de alojamiento te hospedaste durante tu estancia?": "alojamiento",
    "¿Hace cuánto hiciste el Camino?": "hace_cuanto",
    "¿Volverías a hacer el Camino?": "is_volver_a_hacer",
    "¿Cuál crees que es tu tipo ideal de etapa?": "etapa_ideal",
    "¿Qué nota le das a tu experiencia en el Camino? (0–10)": "nota",
    "¿Tuviste alguna lesión o problema físico durante el camino?": "is_lesion",
    "¿Qué consejos darías a alguien como tú antes de hacer el Camino?": "consejos",
    "Medio de transporte": "modo_realizacion",

}

df_original = df_original.rename(columns=reenombre_v)

Conversión de tipos

In [128]:
df_original["edad"] = pd.to_numeric(df_original["edad"].str.extract("(\d+)")[0], errors="coerce")

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-3577741378.py:1: SyntaxWarning: invalid escape sequence '\d'
  df_original["edad"] = pd.to_numeric(df_original["edad"].str.extract("(\d+)")[0], errors="coerce")


In [129]:
df_original["dias"] = pd.to_numeric(
    df_original["dias"], errors="coerce"
)

In [130]:
df_original["distancia"] = pd.to_numeric(
    df_original["distancia"].astype(str).str.extract("(\d+)")[0],
    errors="coerce"
)

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-230473213.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_original["distancia"].astype(str).str.extract("(\d+)")[0],


In [131]:
df_original["n_acompañantes"] = pd.to_numeric(
    df_original["n_acompañantes"].str.extract("(\d+)")[0],
    errors="coerce"
)

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-3944575814.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_original["n_acompañantes"].str.extract("(\d+)")[0],


In [132]:
df_original.head()

,edad,genero,pais,is_condicion_fisica,detalle_condicion,nivel_expereincia_previo,condicion_previa,motivo,n_acompañantes,epoca,...,is_tr_mochila,is_reserva_aloj,alojamiento,hace_cuanto,is_volver_a_hacer,etapa_ideal,nota,is_lesion,consejos,modo_realizacion
0,NaN,Mujer,España,No,NaN,Básica (≤ 10 km ocasionalmente),Media,"Religioso, Deportivo, Cultural/turístico, Pers...",3.0,Verano,...,No,Sí,"Albergues privados, Hostales / Hoteles",Más de 2 años,Sí,Medias (~20 km con algo de desnivel),9,"Sí, leve",NaN,NaN
1,NaN,Hombre,España,NaN,NaN,Avanzada (rutas largas con mochila/montaña),Media,"Deportivo, Cultural/turístico, Personal/reflexivo",3.0,Verano,...,No,Sí,"Albergues públicos, Albergues privados, Hostal...",Más de 2 años,Sí,Medias (~20 km con algo de desnivel),9,No,Hacer etapas parecidas en distancia y terreno ...,NaN
2,NaN,Hombre,España,NaN,NaN,Moderada (10–20 km regularmente),Media,"Religioso, Cultural/turístico",6.0,Primavera,...,Sí,Sí,Hostales / Hoteles,Más de 2 años,Sí,Medias (~20 km con algo de desnivel),8,"Sí, leve",NaN,NaN
3,NaN,Mujer,Colombia,NaN,Migrañas tensionales,Básica (≤ 10 km ocasionalmente),Media,Personal/reflexivo,NaN,Primavera,...,Sí,Sí,"Albergues privados, Hostales / Hoteles",Este año,Sí,Medias (~20 km con algo de desnivel),9,No,NaN,NaN
4,21.0,Mujer,España,NaN,NaN,Básica (≤ 10 km ocasionalmente),Media,"Religioso, Cultural/turístico, Personal/reflexivo",6.0,Verano,...,Sí,Sí,Albergues privados,En los últimos 2 años,Sí,Medias (~20 km con algo de desnivel),10,"Sí, leve",Prefiero no revelar y que cada uno viva su cam...,NaN


Clasificación de variables

In [133]:
v_numericas = ["edad", "dias", "distancia", "nota"]


In [134]:
v_categoricas = ["genero", "pais", "ruta", "epoca",
                    "calzado", "modo_realizacion", "alojamiento"]

In [135]:
v_binarias = ["is_condicion_fisica", "is_calzado_adecuado",
               "is_tr_mochila", "is_reserva_aloj",
               "is_camino_realizado_prev", "is_volver_a_hacer",
               "is_lesion"]

for col in v_binarias:
    df_original[col] = df_original[col].map({"Sí": 1, "Si": 1, "No": 0})


Homegenización del texto

In [136]:
df_original["inicio"].unique()

array(['Sarria', 'O Cebreiro', 'Ferrol', 'Sarrià', 'Sarriá', 'A Guarda',
       'Tui', 'Porrá do Brollon', 'Pontevedra', 'Porto', 'Roncesvalles',
       'Oviedo', 'En Sarria', 'Gijón', 'Caminha', 'Santiago - Fisterra',
       'Sárria', 'Lugo', 'La Guardia', 'Sarrid', 'Ponferrada',
       'Triacastela', 'Vigo', 'Sarría', 'Pamplona', 'Valença', 'Ribadeo',
       'O Porriño', 'Lisboa', 'León', 'Astorga', 'Grado', 'Vilalba',
       'Bilbáo', 'A Coruña', 'Neda', 'Barcelos', 'Tineo', 'Betanzos',
       'Pontedeume', 'Báiona', 'Santander', 'Pola de Allande', 'Bilbao',
       'Saint-Jean-Pied-de-Port', 'Á Coruña', 'A Coruñá', 'Baiona',
       'Bilbaó', 'Burgos', 'Baíona', 'San Sebastián', 'Luarca',
       'Villar de Mazarife', 'A Coruna', 'Logroño', 'Irún', 'A Córuña',
       'A Corúña', 'Bílbao', 'Baióna', 'Baioná'], dtype=object)

In [137]:
df_original["inicio"] = (
    df_original["inicio"]
    .str.strip()
    .str.title()
    .replace({
        "Sarrià": "Sarria",
        "Sarriá": "Sarria",
        "Sarría": "Sarria",
        "Sárria": "Sarria",
        "Sarrid": "Sarria",
        "En Sarria": "Sarria",
        "Sarria ": "Sarria",
        "O Cebreiro": "O Cebreiro",
        "O Cébreiro": "O Cebreiro",
        "Ó Cebreiro": "O Cebreiro",
        "O Cebreiró": "O Cebreiro",
        "O Cebréiro": "O Cebreiro",
        "O cebreiro": "O Cebreiro",
        "A Coruna": "A Coruña",
        "A Coruñá": "A Coruña",
        "Á Coruña": "A Coruña",
        "A Córuña": "A Coruña",
        "A Corúña": "A Coruña",
        "Báiona": "Baiona",
        "Baíona": "Baiona",
        "Baioná": "Baiona",
    })
)

In [138]:
df_original["inicio"].unique()

array(['Sarria', 'O Cebreiro', 'Ferrol', 'A Guarda', 'Tui',
       'Porrá Do Brollon', 'Pontevedra', 'Porto', 'Roncesvalles',
       'Oviedo', 'Gijón', 'Caminha', 'Santiago - Fisterra', 'Lugo',
       'La Guardia', 'Ponferrada', 'Triacastela', 'Vigo', 'Pamplona',
       'Valença', 'Ribadeo', 'O Porriño', 'Lisboa', 'León', 'Astorga',
       'Grado', 'Vilalba', 'Bilbáo', 'A Coruña', 'Neda', 'Barcelos',
       'Tineo', 'Betanzos', 'Pontedeume', 'Baiona', 'Santander',
       'Pola De Allande', 'Bilbao', 'Saint-Jean-Pied-De-Port', 'Bilbaó',
       'Burgos', 'San Sebastián', 'Luarca', 'Villar De Mazarife',
       'Logroño', 'Irún', 'Bílbao', 'Baióna'], dtype=object)

Valores nulos

In [139]:
df_original.isna().sum().sort_values(ascending=False).head(10)

,0
detalle_condicion,1636
n_acompañantes,1292
edad,1002
dias,989
is_condicion_fisica,617
consejos,601
is_volver_a_hacer,125
is_lesion,105
modo_realizacion,47
ruta,2


Explicacion para cada caso

In [140]:
df_original['detalle_condicion'] = df_original['detalle_condicion'].fillna("No")
df_original['is_condicion_fisica'] = df_original['is_condicion_fisica'].fillna("Ninguna")
df_original['n_acompañantes'] = df_original['n_acompañantes'].fillna("Desconocido")
df_original['consejos'] = df_original['consejos'].fillna("Sin respuesta")
df_original['ruta'] = df_original['ruta'].fillna("Desconocido")

In [142]:
df_original['edad'] = df_original['edad'].fillna(df_original['edad'].median())
df_original['dias'] = df_original['dias'].fillna(df_original['dias'].median())
df_original['modo_realizacion'] = df_original['modo_realizacion'].fillna(df_original['modo_realizacion'].mode()[0])
df_original['calzado'] = df_original['calzado'].fillna(df_original['calzado'].mode()[0])

In [143]:
df_original

,edad,genero,pais,is_condicion_fisica,detalle_condicion,nivel_expereincia_previo,condicion_previa,motivo,n_acompañantes,epoca,...,is_tr_mochila,is_reserva_aloj,alojamiento,hace_cuanto,is_volver_a_hacer,etapa_ideal,nota,is_lesion,consejos,modo_realizacion
0,32.0,Mujer,España,0.0,No,Básica (≤ 10 km ocasionalmente),Media,"Religioso, Deportivo, Cultural/turístico, Pers...",3.0,Verano,...,0,1,"Albergues privados, Hostales / Hoteles",Más de 2 años,1.0,Medias (~20 km con algo de desnivel),9,NaN,Sin respuesta,A pie
1,32.0,Hombre,España,Ninguna,No,Avanzada (rutas largas con mochila/montaña),Media,"Deportivo, Cultural/turístico, Personal/reflexivo",3.0,Verano,...,0,1,"Albergues públicos, Albergues privados, Hostal...",Más de 2 años,1.0,Medias (~20 km con algo de desnivel),9,0.0,Hacer etapas parecidas en distancia y terreno ...,A pie
2,32.0,Hombre,España,Ninguna,No,Moderada (10–20 km regularmente),Media,"Religioso, Cultural/turístico",6.0,Primavera,...,1,1,Hostales / Hoteles,Más de 2 años,1.0,Medias (~20 km con algo de desnivel),8,NaN,Sin respuesta,A pie
3,32.0,Mujer,Colombia,Ninguna,Migrañas tensionales,Básica (≤ 10 km ocasionalmente),Media,Personal/reflexivo,Desconocido,Primavera,...,1,1,"Albergues privados, Hostales / Hoteles",Este año,1.0,Medias (~20 km con algo de desnivel),9,0.0,Sin respuesta,A pie
4,21.0,Mujer,España,Ninguna,No,Básica (≤ 10 km ocasionalmente),Media,"Religioso, Cultural/turístico, Personal/reflexivo",6.0,Verano,...,1,1,Albergues privados,En los últimos 2 años,1.0,Medias (~20 km con algo de desnivel),10,NaN,Prefiero no revelar y que cada uno viva su cam...,A pie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,47.0,Mujer,México,0.0,No,Básica (≤ 10 km ocasionalmente),Media,"Cultural/turístico, Deportivo, Religioso",Desconocido,Verano,...,0,0,"Albergues públicos, Albergues privados",Más de 2 años,1.0,Medias (~20 km con algo de desnivel),8,0.0,Etapas de ~20–25 km funcionan bien; cuida la h...,A pie
1996,24.0,Hombre,España,1.0,Fascitis plantar,Moderada (10–20 km regularmente),Media,"Cultural/turístico, Deportivo, Religioso",Desconocido,Verano,...,0,1,"Albergues públicos, Albergues privados",Este año,1.0,Medias (~20 km con algo de desnivel),9,0.0,Etapas de ~20–25 km funcionan bien; cuida la h...,A pie
1997,56.0,Hombre,España,0.0,No,Básica (≤ 10 km ocasionalmente),Alta,"Personal/reflexivo, Religioso",3.0,Verano,...,1,1,Hostales / Hoteles,Este año,NaN,Medias (~20 km con algo de desnivel),7,NaN,Etapas de ~20–25 km funcionan bien; cuida la h...,A pie
1998,32.0,Hombre,España,Ninguna,No,Moderada (10–20 km regularmente),Alta,"Deportivo, Religioso",3.0,Otoño,...,0,0,Albergues privados,En los últimos 2 años,1.0,Medias (~20 km con algo de desnivel),9,0.0,Sin respuesta,A pie


Guardar

In [148]:
df_original.to_csv("Limpieza_Encuesta.csv", sep="\t",index=False)